In [1]:
from app.functions import search, create_load_data
from app.db import DatabaseConnection
from pprint import pprint as pp
from dotenv import load_dotenv 
from app.main import match_img
from app.request import load
from app.constans import *
import pandas as pd
import numpy as np
import shutil
import ijson
import os

conn = DatabaseConnection(**paramsl)
conn.connect()

# load(request_id="e07aaf8f-a587-42af-a0b3-0abd42a7ffc5")

In [3]:
data = search(request_id="e07aaf8f-a587-42af-a0b3-0abd42a7ffc5", input={"brand": "2bme"}, comlumns=["id","product_images"], table_name="origin", conn=conn)

for dt in data:
    print(dt)


(161683, ['j_lr9q2iy31mqdcbgkm7.8b124740fb1ffc739856331eabee5a069ef63ab9.file_lr9spvhwzbew4aeeb.[_].jpg', 'j_lr9q2iy31mqdcbgkm7.8b124740fb1ffc739856331eabee5a069ef63ab9.file_lr9spvhw1tznh8qzie.[_].jpg', 'j_lr9q2iy31mqdcbgkm7.8b124740fb1ffc739856331eabee5a069ef63ab9.file_lr9spvhwjh7u4g1a3.[_].jpg', 'j_lr9q2iy31mqdcbgkm7.8b124740fb1ffc739856331eabee5a069ef63ab9.file_lr9spvhxea644yug0.[_].jpg', 'j_lr9q2iy31mqdcbgkm7.8b124740fb1ffc739856331eabee5a069ef63ab9.file_lr9spvhxp4evp00nz.[_].jpg', 'j_lr9q2iy31mqdcbgkm7.8b124740fb1ffc739856331eabee5a069ef63ab9.file_lr9spvhx173wdvq4bs.[_].jpg'])
(163569, ['j_lr9q2iy31mqdcbgkm7.557290dcffc1d01cfc5d4bf4ab5a82e61731f87c.file_lr9spnm4285jm04gz3.[_].jpg', 'j_lr9q2iy31mqdcbgkm7.557290dcffc1d01cfc5d4bf4ab5a82e61731f87c.file_lr9spnm42dk9emrtlm.[_].jpg', 'j_lr9q2iy31mqdcbgkm7.557290dcffc1d01cfc5d4bf4ab5a82e61731f87c.file_lr9spnm495io8x9jf.[_].jpg', 'j_lr9q2iy31mqdcbgkm7.557290dcffc1d01cfc5d4bf4ab5a82e61731f87c.file_lr9spnm4t8jvgdgn6.[_].jpg', 'j_lr9q2iy31mqd

In [2]:
request = {
    "request_id": "e07aaf8f-a587-42af-a0b3-0abd42a7ffc5",
    "input": {
        "origin":{"brand": "2bme"},
        "alternative":{"brand": "2Bme"},
    },
    "s3_path_img_origin": "ajio-myntra/origin/20240112/",
    "s3_path_img_alternative": "ajio-myntra/alternative/20240110/"
    
}

try: 
    match_img(**request)
except TypeError as e:

    path = "trash"
    for name in ["data", "fastdup", "s3", "reports", "img", "db"]:
        path_local = os.path.join(path, name)
        shutil.rmtree(path_local)
        os.makedirs(path_local)
    
    print(e)
    
finally:

    path = "trash"
    for name in ["data", "fastdup", "s3", "reports", "img", "db"]:
        path_local = os.path.join(path, name)
        shutil.rmtree(path_local)
        os.makedirs(path_local)

(161683, ['j_lr9q2iy31mqdcbgkm7.8b124740fb1ffc739856331eabee5a069ef63ab9.file_lr9spvhwzbew4aeeb.[_].jpg', 'j_lr9q2iy31mqdcbgkm7.8b124740fb1ffc739856331eabee5a069ef63ab9.file_lr9spvhw1tznh8qzie.[_].jpg', 'j_lr9q2iy31mqdcbgkm7.8b124740fb1ffc739856331eabee5a069ef63ab9.file_lr9spvhwjh7u4g1a3.[_].jpg', 'j_lr9q2iy31mqdcbgkm7.8b124740fb1ffc739856331eabee5a069ef63ab9.file_lr9spvhxea644yug0.[_].jpg', 'j_lr9q2iy31mqdcbgkm7.8b124740fb1ffc739856331eabee5a069ef63ab9.file_lr9spvhxp4evp00nz.[_].jpg', 'j_lr9q2iy31mqdcbgkm7.8b124740fb1ffc739856331eabee5a069ef63ab9.file_lr9spvhx173wdvq4bs.[_].jpg'])
(2058, ['j_lr7t5v3x1ag82zisab.f6ab8d42897a854f32f25876a3556226e267cb4b.file_lr7u45946oxp28tt0.[_].webp', 'j_lr7t5v3x1ag82zisab.f6ab8d42897a854f32f25876a3556226e267cb4b.file_lr7u45941mpypo9hby.[_].webp', 'j_lr7t5v3x1ag82zisab.f6ab8d42897a854f32f25876a3556226e267cb4b.file_lr7u4594hn83lhulj.[_].jpg', 'j_lr7t5v3x1ag82zisab.f6ab8d42897a854f32f25876a3556226e267cb4b.file_lr7u45941vii5xhduw.[_].webp'])
